# 소셜 버즈 데이터 수집 및 분류
[목표]
- poi(관광지)별 2021년 날짜에 해당하는 소셜 버즈 데이터를 수집한 후, poi별 데이터셋 작성

[진행할 과정]
1. 네이버 블로그 크롤링
2. 크롤링 데이터 분류

[주의사항]
- 크롤링을 돌릴 때, social buzz 폴더안의 모든 데이터를 삭제하고 돌리셔야 합니다.

## 1차 : 네이버 블로그 크롤링
- 과정 1 : 2021년 모든 날짜를 리스트로 저장
- 과정 2 : poi리스트를 가져와 한 poi당 당 모든 요일 수인 365번 수행 (1.1~12.31)
- 과정 3 : 약 200개의 모든 poi들에 대해 과정 2 수행

[데이터셋]
- 관광지 : 제주도 내의 poi(관광지)의 명칭 저장.

[변수 설명]
1. 기본 변수
- poi : 각 poi(관광지 데이터)
- date_list : 2021-01-01 ~ 2021-12-31 사이의 모든 날짜들의 리스트
- search : 특정 관광지 지정 (크롤링 내의 반복문에서 활용)

2. url 변수
- url_first : startdate를 입력하기 전 url
- url_second : startdate를 입력한 이후 enddate를 입력하기 이전 url
- url_third : enddate 입력 이후, keyword, 즉 poi를 입력하기 이전 url

3. 크롤링 변수
- visit_list : poi명 (관광지명)을 리스트 형태로 나타냄 (1개의 관광지당 365번)
- count_list : 소셜 버즈량(1월1일부터 12월31일까지)
- count_num : 관광지 해당 요일의 소셜 버즈량을 int형태로 나타낸 값
- result : 관광지, 소셜 버즈량, 날짜 데이터를 모두 가져와 합친 데이터프레임

[크롤링 과정]
- 네이버 블로그 사이트에서, 각 관광지 별 2021년 1월 1일부터 2021년 12월 31일까지 일마다 몇개의 블로그 개수가 있는지 크롤링 후 데이터프레임으로 저장 -> 소셜버즈량의 데이터로 사용

- 작동 원리 : 하나의 관광지별로 1월1일부터 12월31일까지의 일별 개수를 크롤링한 다음, 다음 관광지로 넘기기

[결과 데이터셋]
- 관광지명 : 1월1일부터 12월31일까지의 개수인 365개의 데이터가 각 관광지별로 존재( ex. 거슨세미가 365개, 헬로키티아일랜드도 365개)
- 날짜 : 각 요일, 2021-01-01부터 2021-12-31까지 각 관광지 별 365개 존재
- 소셜버즈량 수 : 각 관광지별, 일별로 몇건의 블로그 개수가 있는지 결과

# 2차 : 크롤링 데이터 분류

[전처리 과정]

- 크롤링한 관광지들의 소셜 버즈량 데이터를 모두 갖고와 합침 
    - 총 121개의 관광지 데이터가 존재하기 때문에, 한번에 121개의 모든 데이터를 합치는 데에는 무리가 있으므로 10개씩 합침.
- 기존의 poi 설명이 적힌 데이터(Jeju_POI_withexplanation_final.xlsx)도 가져와 이와 비교한 후, 이 관광지의 시를 구분하여 새로운른 폴더에 새로 저장하기

[변수 설명]
- a1~a0 : 10개씩 합칠 각 poi별 데이터 (코드 진행할수록 갱신해서 반복수행)
- b1~b0 : a1~a0을 합친 데이터셋 (코드 진행할수록 갱신해서 반복수행, 단 b1,b2만)
- total : 모든 poi별 데이터를 합친 결과 데이터프레임
- df : search가 각 poi에 해당하는 데이터프레임
- final : 누락된 값 없이 모두 합친 total 데이터프레임

[데이터 칼럼]

- 관광지 : poi데이터에 있는 제주도의 관광지
- 날짜 : 2021-01-01 ~ 2021-12-31 지정날짜
- 버즈량 : 네이버 블로그의 해당 관광지의 해당 일자에 있는 게시글의 수

In [ ]:
!pip install selenium
!pip install beautifulsoup

In [27]:
# 라이브러리 불러오기
from bs4 import BeautifulSoup
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains

In [6]:
# poi 데이터 가져오기
poi = pd.read_excel('POIfinal.xlsnA')
poi.columns=['관광지']
poi

,관광지
0,헬로키티아일랜드
1,엉알해안산책로
2,1100고지습지
3,우도도항선선착장
4,혼인지
...,...
116,제주불빛정원
117,더마파크
118,거문오름
119,다랑쉬오름


In [7]:
# 2021-01-01에서 2021-12-31 사이 모든 날짜를 크롤링하는 함수 만들어서 가져오기
from datetime import datetime, timedelta
# 해당 범위 내의 날짜들을 리스트로 지정해주는 함수
def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates
# 2021-01-01 ~ 2021-12-31 사이의 날짜를 리스트로 지정
date_list = date_range("2021-01-01", "2021-12-31")

In [8]:
# date_list는 str형식이기 때문에 크롤링 과정에서 굳이 바꿔줄 필요 없음
date_list

['2021-01-01',
 '2021-01-02',
 '2021-01-03',
 '2021-01-04',
 '2021-01-05',
 '2021-01-06',
 '2021-01-07',
 '2021-01-08',
 '2021-01-09',
 '2021-01-10',
 '2021-01-11',
 '2021-01-12',
 '2021-01-13',
 '2021-01-14',
 '2021-01-15',
 '2021-01-16',
 '2021-01-17',
 '2021-01-18',
 '2021-01-19',
 '2021-01-20',
 '2021-01-21',
 '2021-01-22',
 '2021-01-23',
 '2021-01-24',
 '2021-01-25',
 '2021-01-26',
 '2021-01-27',
 '2021-01-28',
 '2021-01-29',
 '2021-01-30',
 '2021-01-31',
 '2021-02-01',
 '2021-02-02',
 '2021-02-03',
 '2021-02-04',
 '2021-02-05',
 '2021-02-06',
 '2021-02-07',
 '2021-02-08',
 '2021-02-09',
 '2021-02-10',
 '2021-02-11',
 '2021-02-12',
 '2021-02-13',
 '2021-02-14',
 '2021-02-15',
 '2021-02-16',
 '2021-02-17',
 '2021-02-18',
 '2021-02-19',
 '2021-02-20',
 '2021-02-21',
 '2021-02-22',
 '2021-02-23',
 '2021-02-24',
 '2021-02-25',
 '2021-02-26',
 '2021-02-27',
 '2021-02-28',
 '2021-03-01',
 '2021-03-02',
 '2021-03-03',
 '2021-03-04',
 '2021-03-05',
 '2021-03-06',
 '2021-03-07',
 '2021-03-

In [9]:
# 전체 date_list의 길이 확인
len(date_list)

365

In [10]:
# 테스트 : 101번째 관광지가 무엇인지 출력
search = poi['관광지'][100]
search

'한라생태숲'

소요 : 365 * 210 = 76650

## 크롤링 코드

In [42]:
# 크롤링 코드
search = poi['관광지'][0]
url_first = 'https://section.blog.naver.com/Search/Post.naver?pageNo=' # 페이지 입력전 첫번째 URL 부분
url_second = '&rangeType=PERIOD&orderBy=recentdate&startDate=2021-01-01&endDate=2021-12-31&keyword=' # 페이지 입력 후, 관광지 입력 전 URL 부분
url = url_first + '1' + url_second + search # 해당 관광지명의 1페이지로 검색
print(url)
driver = webdriver.Chrome() # 드라이버 열기
driver.get(url)

# URL을 세부분으로 나눠 시작날짜를 입력하는 곳이 첫번째, 끝날짜가 2번째, 관광지가 3번째
url_first = 'https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate='
url_second = '&endDate='
url_third = '&keyword='
# 첫번째 for문에 숫자 지정하면 지정하는 대로의 관광지들을 싹 크롤링해서 저장합니다.
# 만약 오류가 발생하면 너무 많이 돌려서 그런 것이여서, 오류난 지점부터 다시 돌리면 됩니다.

# 중첩 반복문 구조 : 관광지별 2021.01.01~2021.12.31까지 크롤링 완료 후 다음 관광지로 넘어가기
# 첫번째 조건 : 관광지별 -> 하나의 관광지별로 2021년의 모든 날짜의 크롤링 값 가져오기
for j in range(len(poi['관광지'])):
    search = poi['관광지'][j]
    visit_list = [] # 관광지 리스트
    count_list = [] # 소셜버즈량 리스트
    
    # 2번째 조건 : 일자별 (2021.1.1 ~ 2021.12.31)
    for i in range(len(date_list)):
        # 분할시킨 url을 일자, 관광지를 합쳐 url 완성
        url = url_first + date_list[i] + url_second + date_list[i] + url_third + search
    #     driver = webdriver.Chrome() # 드라이버 열기
        driver.get(url)
        time.sleep(1.5) # 1.5초 time sleep
        count = 0
        try : 
            count = driver.find_element(By.XPATH, '//*[@id="content"]/section/div[1]/div[2]/span/span/em')
        except : # 잘 못찾는 경우가 있어서 except에도 같은 경우를 넣어줌. 이렇게 되면 못찾는 일이 없음
            count = driver.find_element(By.XPATH, '//*[@id="content"]/section/div[1]/div[2]/span/span/em')
        finally : 
            # 몇건이 조회되었는지 가져옴
            count_num = count.text # 텍스트만 추출
            count_num = count_num.replace(',','') # ,을 빈칸으로 변환
            count_num = int(count_num.split('건')[0]) # 앞의 숫자만 가져오고 형태 변환
            count_list.append(count_num) # count_list에 추가
            visit_list.append(search) # visit_list에 일자 수만큼 관광지명 넣기
            print(date_list[i] + search) # 하나의 관광지에 해당 요일에 대한 크롤링이 완료되었을 때 출력
    result = pd.DataFrame(visit_list, count_list) # 데이터프레임으로
    result['날짜'] = date_list # 날짜 칼럼 집어넣기
    result.reset_index(inplace=True) # 인덱스 초기화
    result.columns = ['버즈량','관광지','날짜'] # 칼럼명 재지정
    result = result[['관광지','날짜','버즈량']] # 칼럼 순서 조정
    # 파일명 지정하거나 원하는 위치에 부탁...
    result.to_csv('./Dataset/rawdata/social buzz/' + search + ' 소셜버즈량.csv',encoding='utf-8-sig') # poi별 따로 저장
    print('{:d}번째관광지 저장 완료'.format(j+1))# 저장되었으면, n번째관광지 저장 완료라고 표시

https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=recentdate&startDate=2021-01-01&endDate=2021-12-31&keyword=헬로키티아일랜드
2021-01-01한라생태숲
2021-01-02한라생태숲
2021-01-03한라생태숲
2021-01-04한라생태숲
2021-01-05한라생태숲
2021-01-06한라생태숲
2021-01-07한라생태숲
2021-01-08한라생태숲
2021-01-09한라생태숲
2021-01-10한라생태숲
2021-01-11한라생태숲
2021-01-12한라생태숲
2021-01-13한라생태숲
2021-01-14한라생태숲
2021-01-15한라생태숲
2021-01-16한라생태숲
2021-01-17한라생태숲
2021-01-18한라생태숲
2021-01-19한라생태숲
2021-01-20한라생태숲
2021-01-21한라생태숲
2021-01-22한라생태숲
2021-01-23한라생태숲
2021-01-24한라생태숲
2021-01-25한라생태숲
2021-01-26한라생태숲
2021-01-27한라생태숲
2021-01-28한라생태숲
2021-01-29한라생태숲
2021-01-30한라생태숲
2021-01-31한라생태숲
2021-02-01한라생태숲
2021-02-02한라생태숲
2021-02-03한라생태숲
2021-02-04한라생태숲
2021-02-05한라생태숲
2021-02-06한라생태숲
2021-02-07한라생태숲
2021-02-08한라생태숲
2021-02-09한라생태숲
2021-02-10한라생태숲
2021-02-11한라생태숲
2021-02-12한라생태숲
2021-02-13한라생태숲
2021-02-14한라생태숲
2021-02-15한라생태숲
2021-02-16한라생태숲
2021-02-17한라생태숲
2021-02-18한라생태숲
2021-02-19한라생태숲
2021-02-20한라생태숲
2021-02-21한라생태숲
2021-02-22한라생태숲
2

In [20]:
len(count_list), len(visit_list), len(dates)

(365, 365, 365)

# 2. 크롤링한 데이터 합치기
- 10개씩 합친 다음, 이어합치기

In [28]:
# 데이터 합치기 #1
a1 = pd.read_csv('../Dataset/raw_data/social buzz/1100고지습지 소셜버즈량.csv')
a2 = pd.read_csv('../Dataset/raw_data/social buzz/가파도 소셜버즈량.csv')
a3 = pd.read_csv('../Dataset/raw_data/social buzz/건강과성박물관 소셜버즈량.csv')
a4 = pd.read_csv('../Dataset/raw_data/social buzz/걸매생태공원 소셜버즈량.csv')
a5 = pd.read_csv('../Dataset/raw_data/social buzz/곽지해수욕장 소셜버즈량.csv')
a6 = pd.read_csv('../Dataset/raw_data/social buzz/관덕정 소셜버즈량.csv')
a7 = pd.read_csv('../Dataset/raw_data/social buzz/광치기해변 소셜버즈량.csv')
a8 = pd.read_csv('../Dataset/raw_data/social buzz/국립제주박물관 소셜버즈량.csv')
a9 = pd.read_csv('../Dataset/raw_data/social buzz/군산오름 소셜버즈량.csv')
a0 = pd.read_csv('../Dataset/raw_data/social buzz/그리스신화박물관 소셜버즈량.csv')

b1 = pd.concat([a1,a2,a3,a4,a5,a6,a7,a8,a9,a0])
b1['관광지'].nunique()

10

In [29]:
# 데이터 합치기 #2
a1 = pd.read_csv('../Dataset/raw_data/social buzz/금능석물원 소셜버즈량.csv')
a2 = pd.read_csv('../Dataset/raw_data/social buzz/금오름 소셜버즈량.csv')
a3 = pd.read_csv('../Dataset/raw_data/social buzz/김녕미로공원 소셜버즈량.csv')
a4 = pd.read_csv('../Dataset/raw_data/social buzz/김녕해수욕장 소셜버즈량.csv')
a5 = pd.read_csv('../Dataset/raw_data/social buzz/김영갑갤러리두모악 소셜버즈량.csv')
a6 = pd.read_csv('../Dataset/raw_data/social buzz/남원큰엉 소셜버즈량.csv')
a7 = pd.read_csv('../Dataset/raw_data/social buzz/노루생태관찰원 소셜버즈량.csv')
a8 = pd.read_csv('../Dataset/raw_data/social buzz/동문재래시장 소셜버즈량.csv')
a9 = pd.read_csv('../Dataset/raw_data/social buzz/동백동산 소셜버즈량.csv')
a0 = pd.read_csv('../Dataset/raw_data/social buzz/따라비오름 소셜버즈량.csv')

b2 = pd.concat([a1,a2,a3,a4,a5,a6,a7,a8,a9,a0])
b2['관광지'].nunique()

10

In [30]:
# 데이터 합치기 #3
a1 = pd.read_csv('../Dataset/raw_data/social buzz/만장굴 소셜버즈량.csv')
a2 = pd.read_csv('../Dataset/raw_data/social buzz/메이즈랜드 소셜버즈량.csv')
a3 = pd.read_csv('../Dataset/raw_data/social buzz/물영아리오름 소셜버즈량.csv')
a4 = pd.read_csv('../Dataset/raw_data/social buzz/박물관은 살아있다  소셜버즈량.csv')
a5 = pd.read_csv('../Dataset/raw_data/social buzz/본태박물관 소셜버즈량.csv')
a6 = pd.read_csv('../Dataset/raw_data/social buzz/붉은오름자연휴양림 소셜버즈량.csv')
a7 = pd.read_csv('../Dataset/raw_data/social buzz/비양도 소셜버즈량.csv')
a8 = pd.read_csv('../Dataset/raw_data/social buzz/비자림 소셜버즈량.csv')
a9 = pd.read_csv('../Dataset/raw_data/social buzz/사진놀이터 소셜버즈량.csv')
a0 = pd.read_csv('../Dataset/raw_data/social buzz/산굼부리 소셜버즈량.csv')

b3 = pd.concat([a1,a2,a3,a4,a5,a6,a7,a8,a9,a0])
b3['관광지'].nunique()

10

In [31]:
# 데이터 합치기 #4
a1 = pd.read_csv('../Dataset/raw_data/social buzz/산방굴사 소셜버즈량.csv')
a2 = pd.read_csv('../Dataset/raw_data/social buzz/산방산 소셜버즈량.csv')
a3 = pd.read_csv('../Dataset/raw_data/social buzz/산방산탄산온천 소셜버즈량.csv')
a4 = pd.read_csv('../Dataset/raw_data/social buzz/삼성혈 소셜버즈량.csv')
a5 = pd.read_csv('../Dataset/raw_data/social buzz/삼양해수욕장 소셜버즈량.csv')
a6 = pd.read_csv('../Dataset/raw_data/social buzz/생각하는 정원 소셜버즈량.csv')
a7 = pd.read_csv('../Dataset/raw_data/social buzz/서귀포매일올레시장 소셜버즈량.csv')
a8 = pd.read_csv('../Dataset/raw_data/social buzz/서귀포잠수함 소셜버즈량.csv')
a9 = pd.read_csv('../Dataset/raw_data/social buzz/선녀와나무꾼테마공원 소셜버즈량.csv')
a0 = pd.read_csv('../Dataset/raw_data/social buzz/섭지코지 소셜버즈량.csv')

b4 = pd.concat([a1,a2,a3,a4,a5,a6,a7,a_8,a9,a0])
b4['관광지'].nunique()

10

In [32]:
# 데이터 합치기 #5
a1 = pd.read_csv('../Dataset/raw_data/social buzz/성산일출봉 소셜버즈량.csv')
a2 = pd.read_csv('../Dataset/raw_data/social buzz/성읍민속마을 소셜버즈량.csv')
a3 = pd.read_csv('../Dataset/raw_data/social buzz/세화해수욕장 소셜버즈량.csv')
a4 = pd.read_csv('../Dataset/raw_data/social buzz/소정방폭포 소셜버즈량.csv')
a5 = pd.read_csv('../Dataset/raw_data/social buzz/쇠소깍 소셜버즈량.csv')
a6 = pd.read_csv('../Dataset/raw_data/social buzz/수월봉 소셜버즈량.csv')
a7 = pd.read_csv('../Dataset/raw_data/social buzz/신양섭지해수욕장 소셜버즈량.csv')
a8 = pd.read_csv('../Dataset/raw_data/social buzz/아부오름 소셜버즈량.csv')
a9 = pd.read_csv('../Dataset/raw_data/social buzz/아쿠아플라넷제주 소셜버즈량.csv')
a0 = pd.read_csv('../Dataset/raw_data/social buzz/안덕계곡 소셜버즈량.csv')

b5 = pd.concat([a1,a2,a3,a4,a5,a6,a7,a8,a9,a0])
b5['관광지'].nunique()

10

In [33]:
# 데이터 합치기 #6
a1 = pd.read_csv('../Dataset/raw_data/social buzz/알뜨르비행장 소셜버즈량.csv')
a2 = pd.read_csv('../Dataset/raw_data/social buzz/약천사 소셜버즈량.csv')
a3 = pd.read_csv('../Dataset/raw_data/social buzz/어승생악 소셜버즈량.csv')
a4 = pd.read_csv('../Dataset/raw_data/social buzz/엉또폭포 소셜버즈량.csv')
a5 = pd.read_csv('../Dataset/raw_data/social buzz/엉알해안산책로 소셜버즈량.csv')
a6 = pd.read_csv('../Dataset/raw_data/social buzz/에코랜드테마파크 소셜버즈량.csv')
a7 = pd.read_csv('../Dataset/raw_data/social buzz/여미지식물원 소셜버즈량.csv')
a8 = pd.read_csv('../Dataset/raw_data/social buzz/외돌개 소셜버즈량.csv')
a9 = pd.read_csv('../Dataset/raw_data/social buzz/용두암 소셜버즈량.csv')
a0 = pd.read_csv('../Dataset/raw_data/social buzz/용머리해안 소셜버즈량.csv')

b6 = pd.concat([a1,a2,a3,a4,a5,a6,a7,a8,a9,a0])
b6['관광지'].nunique()

10

In [34]:
# 데이터 합치기 #7
a1 = pd.read_csv('../Dataset/raw_data/social buzz/우도도항선선착장 소셜버즈량.csv')
a2 = pd.read_csv('../Dataset/raw_data/social buzz/우도잠수함 소셜버즈량.csv')
a3 = pd.read_csv('../Dataset/raw_data/social buzz/원앙폭포 소셜버즈량.csv')
a4 = pd.read_csv('../Dataset/raw_data/social buzz/월정리해변 소셜버즈량.csv')
a5 = pd.read_csv('../Dataset/raw_data/social buzz/이호테우해변 소셜버즈량.csv')
a6 = pd.read_csv('../Dataset/raw_data/social buzz/일출랜드 소셜버즈량.csv')
a7 = pd.read_csv('../Dataset/raw_data/social buzz/점보빌리지 소셜버즈량.csv')
a8 = pd.read_csv('../Dataset/raw_data/social buzz/정물오름 소셜버즈량.csv')
a9 = pd.read_csv('../Dataset/raw_data/social buzz/정방폭포 소셜버즈량.csv')
a0 = pd.read_csv('../Dataset/raw_data/social buzz/제이엠그랑블루요트 소셜버즈량.csv')

b7 = pd.concat([a1,a2,a3,a4,a5,a6,a7,a8,a9,a0])
b7['관광지'].nunique()

10

In [35]:
# 데이터 합치기 #8
a1 = pd.read_csv('../Dataset/raw_data/social buzz/제주 초콜릿랜드 소셜버즈량.csv')
a2 = pd.read_csv('../Dataset/raw_data/social buzz/제주4∙3평화공원 소셜버즈량.csv')
a3 = pd.read_csv('../Dataset/raw_data/social buzz/제주공룡랜드 소셜버즈량.csv')
a4 = pd.read_csv('../Dataset/raw_data/social buzz/제주다원녹차미로공원 소셜버즈량.csv')
a5 = pd.read_csv('../Dataset/raw_data/social buzz/제주돌문화공원 소셜버즈량.csv')
a6 = pd.read_csv('../Dataset/raw_data/social buzz/제주민속자연사박물관 소셜버즈량.csv')
a7 = pd.read_csv('../Dataset/raw_data/social buzz/제주민속촌 소셜버즈량.csv')
a8 = pd.read_csv('../Dataset/raw_data/social buzz/제주절물자연휴양림 소셜버즈량.csv')
a9 = pd.read_csv('../Dataset/raw_data/social buzz/제주조각공원 소셜버즈량.csv')
a0 = pd.read_csv('../Dataset/raw_data/social buzz/제주해녀박물관 소셜버즈량.csv')

b8 = pd.concat([a1,a2,a3,a4,a5,a6,a7,a8,a9,a0])
b8['관광지'].nunique()

10

In [36]:
# 데이터 합치기 #9
a1 = pd.read_csv('../Dataset/raw_data/social buzz/제주허브동산 소셜버즈량.csv')
a2 = pd.read_csv('../Dataset/raw_data/social buzz/조랑말체험공원 소셜버즈량.csv')
a3 = pd.read_csv('../Dataset/raw_data/social buzz/종달리해안도로 소셜버즈량.csv')
a4 = pd.read_csv('../Dataset/raw_data/social buzz/주상절리대 소셜버즈량.csv')
a5 = pd.read_csv('../Dataset/raw_data/social buzz/중문관광단지 소셜버즈량.csv')
a6 = pd.read_csv('../Dataset/raw_data/social buzz/지미봉 소셜버즈량.csv')
a7 = pd.read_csv('../Dataset/raw_data/social buzz/천제연폭포 소셜버즈량.csv')
a8 = pd.read_csv('../Dataset/raw_data/social buzz/천지연폭포  소셜버즈량.csv')
a9 = pd.read_csv('../Dataset/raw_data/social buzz/초콜릿박물관 소셜버즈량.csv')
a0 = pd.read_csv('../Dataset/raw_data/social buzz/테디베어뮤지엄제주 소셜버즈량.csv')

b9 = pd.concat([a1,a2,a3,a4,a5,a6,a7,a8,a9,a0])
b9['관광지'].nunique()

10

In [37]:
# 데이터 합치기 #10
a1 = pd.read_csv('../Dataset/raw_data/social buzz/퍼시픽리솜 소셜버즈량.csv')
a2 = pd.read_csv('../Dataset/raw_data/social buzz/피규어뮤지엄제주 소셜버즈량.csv')
a3 = pd.read_csv('../Dataset/raw_data/social buzz/한라산 소셜버즈량.csv')
a4 = pd.read_csv('../Dataset/raw_data/social buzz/한라수목원 소셜버즈량.csv')
a5 = pd.read_csv('../Dataset/raw_data/social buzz/한림공원 소셜버즈량.csv')
a6 = pd.read_csv('../Dataset/raw_data/social buzz/헬로키티아일랜드 소셜버즈량.csv')
a7 = pd.read_csv('../Dataset/raw_data/social buzz/협재해수욕장 소셜버즈량.csv')
a8 = pd.read_csv('../Dataset/raw_data/social buzz/혼인지 소셜버즈량.csv')
a9 = pd.read_csv('../Dataset/raw_data/social buzz/화순금모래 해변 소셜버즈량.csv')
a0 = pd.read_csv('../Dataset/raw_data/social buzz/황우지해안 소셜버즈량.csv')

b0 = pd.concat([a1,a2,a3,a4,a5,a6,a7,a8,a9,a0])
b0['관광지'].nunique()

10

In [38]:
# 기존에 했던 100개 데이터를 합침
total = pd.concat([b1,b2,b3,b4,b5,b6,b7,b8,b9,b0])

In [39]:
# 까먹었던 한라생태숲 소셜 버즈량까지 total 데이터에 합치기
a1 = pd.read_csv('../Dataset/raw_data/social buzz/한라생태숲 소셜버즈량.csv')
total = pd.concat([total, a1])

In [40]:
# 관광지 개수 확인
total['관광지'].nunique()

101

In [41]:
#데이터 합치기 #11
a1 = pd.read_csv('../Dataset/raw_data/social buzz/거문오름 소셜버즈량.csv')
a2 = pd.read_csv('../Dataset/raw_data/social buzz/검멀레해수욕장 소셜버즈량.csv')
a3 = pd.read_csv('../Dataset/raw_data/social buzz/금능해수욕장 소셜버즈량.csv')
a4 = pd.read_csv('../Dataset/raw_data/social buzz/다랑쉬오름 소셜버즈량.csv')
a5 = pd.read_csv('../Dataset/raw_data/social buzz/더마파크 소셜버즈량.csv')
a6 = pd.read_csv('../Dataset/raw_data/social buzz/브릭캠퍼스제주 소셜버즈량.csv')
a7 = pd.read_csv('../Dataset/raw_data/social buzz/새별오름 소셜버즈량.csv')
a8 = pd.read_csv('../Dataset/raw_data/social buzz/서귀포치유의숲 소셜버즈량.csv')
a9 = pd.read_csv('../Dataset/raw_data/social buzz/세계자동차&피아노박물관 소셜버즈량.csv')
a0 = pd.read_csv('../Dataset/raw_data/social buzz/신창풍차해안도로 소셜버즈량.csv')

b1 = pd.concat([a1,a2,a3,a4,a5,a6,a7,a8,a9,a0])
b1

,Unnamed: 0,관광지,날짜,버즈량
0,0,거문오름,2021-01-01,6
1,1,거문오름,2021-01-02,14
2,2,거문오름,2021-01-03,11
3,3,거문오름,2021-01-04,10
4,4,거문오름,2021-01-05,15
...,...,...,...,...
360,360,신창풍차해안도로,2021-12-27,31
361,361,신창풍차해안도로,2021-12-28,20
362,362,신창풍차해안도로,2021-12-29,22
363,363,신창풍차해안도로,2021-12-30,22


In [42]:
# 데이터 합치기 #12
a1 = pd.read_csv('../Dataset/raw_data/social buzz/용눈이오름 소셜버즈량.csv')
a2 = pd.read_csv('../Dataset/raw_data/social buzz/제주별빛누리공원 소셜버즈량.csv')
a3 = pd.read_csv('../Dataset/raw_data/social buzz/제주불빛정원 소셜버즈량.csv')
a4 = pd.read_csv('../Dataset/raw_data/social buzz/제주신산공원 소셜버즈량.csv')
a5 = pd.read_csv('../Dataset/raw_data/social buzz/제주유리의 성 소셜버즈량.csv')
a6 = pd.read_csv('../Dataset/raw_data/social buzz/제주항공우주박물관 소셜버즈량.csv')
a7 = pd.read_csv('../Dataset/raw_data/social buzz/제주현대미술관 소셜버즈량.csv')
a8 = pd.read_csv('../Dataset/raw_data/social buzz/카멜리아힐 소셜버즈량.csv')
a9 = pd.read_csv('../Dataset/raw_data/social buzz/화조원 소셜버즈량.csv')
a0 = pd.read_csv('../Dataset/raw_data/social buzz/휴애리자연생활공원 소셜버즈량.csv')

b2 = pd.concat([a1,a2,a3,a4,a5,a6,a7,a8,a9,a0])
b2

,Unnamed: 0,관광지,날짜,버즈량
0,0,용눈이오름,2021-01-01,15
1,1,용눈이오름,2021-01-02,28
2,2,용눈이오름,2021-01-03,22
3,3,용눈이오름,2021-01-04,18
4,4,용눈이오름,2021-01-05,21
...,...,...,...,...
360,360,휴애리자연생활공원,2021-12-27,6
361,361,휴애리자연생활공원,2021-12-28,4
362,362,휴애리자연생활공원,2021-12-29,6
363,363,휴애리자연생활공원,2021-12-30,10


In [43]:
# 모든 데이터 합치기
total = pd.concat([total,b1,b2])
total['관광지'].nunique()

121

In [44]:
# 불필요 칼럼 제거
total.drop('Unnamed: 0',axis=1,inplace=True)

In [45]:
# poi설명 적혀있는 데이터 호출, 필요없는 칼럼 제거
poi2 = pd.read_excel('../Dataset/final_data/Jeju_POI_withexplanation_final.xlsx')
poi2.drop('Unnamed: 0',axis=1,inplace=True)

In [46]:
# 칼럼명 재지정
poi2.columns = ['관광지','위치','해시태그']

In [47]:
# 관광지, 위치 칼럼만 남겨놓기
poi2 = poi2[['관광지','위치']]

In [48]:
# 관광지 칼럼을 기준으로 total, poi2합치기
final = pd.merge(total, poi2, on = '관광지')

In [50]:
# 합친 데이터 저장
final.to_csv('../Dataset/raw_data/social buzz/social buzz final.csv',encoding='utf-8-sig',index=False)

In [53]:
final['관광지'].nunique()

121

In [54]:
final_jeju = final[final['위치']=='제주시'] # 위치가 제주시인 관광지
final_seogwipo = final[final['위치']=='서귀포시'] # 위치가 서귀포시인 관광지

In [55]:
# 파일들 저장
final_jeju.to_csv('../Dataset/raw_data/social buzz/jejusi_final.csv',encoding='utf-8-sig')
final_seogwipo.to_csv('../Dataset/raw_data/social buzz/seogwipo_final.csv',encoding='utf-8-sig')

In [56]:
# 제대로 나와있는지 확인
final['관광지'].value_counts()

1100고지습지     365
우도잠수함        365
테디베어뮤지엄제주    365
초콜릿박물관       365
천지연폭포        365
            ... 
서귀포매일올레시장    365
생각하는 정원      365
삼양해수욕장       365
삼성혈          365
휴애리자연생활공원    365
Name: 관광지, Length: 121, dtype: int64

In [57]:
# 관광지별로 나눠서 저장
# 전체 실행할 횟수 : final 데이터의 관광지의 중복되지 않는 개수(총 관광지의 개수)
# 기존의 크롤링 했던 데이터에서 혹시 모를 상황을 방지하기 위해 다른 폴더에 저장했음
for i in range(final['관광지'].nunique()):
    search = final['관광지'].unique()[i] # 관광지명 
    print(search) # 관광지명 프린트
    df = final[final['관광지']==search] # 관광지가 지정한 관광지(search)인 데이터만 가져오기
    df.to_csv('../Dataset/raw_data/social buzz/{:s} 소셜버즈량.csv'.format(search), index=False, encoding='utf-8-sig')

1100고지습지
가파도
건강과성박물관
걸매생태공원
곽지해수욕장
관덕정
광치기해변
국립제주박물관
군산오름
그리스신화박물관
금능석물원
금오름
김녕미로공원
김녕해수욕장
김영갑갤러리두모악
남원큰엉
노루생태관찰원
동문재래시장
동백동산
따라비오름
만장굴
메이즈랜드
물영아리오름
박물관은 살아있다 
본태박물관
붉은오름자연휴양림
비양도
비자림
사진놀이터
산굼부리
산방굴사
산방산
산방산탄산온천
삼성혈
삼양해수욕장
생각하는 정원
서귀포매일올레시장
서귀포잠수함
선녀와나무꾼테마공원
섭지코지
성산일출봉
성읍민속마을
세화해수욕장
소정방폭포
쇠소깍
수월봉
신양섭지해수욕장
아부오름
아쿠아플라넷제주
안덕계곡
알뜨르비행장
약천사
어승생악
엉또폭포
엉알해안산책로
에코랜드테마파크
여미지식물원
외돌개
용두암
용머리해안
우도도항선선착장
우도잠수함
원앙폭포
월정리해변
이호테우해변
일출랜드
점보빌리지
정물오름
정방폭포
제이엠그랑블루요트
제주 초콜릿랜드
제주4∙3평화공원
제주공룡랜드
제주다원녹차미로공원
제주돌문화공원
제주민속자연사박물관
제주민속촌
제주절물자연휴양림
제주조각공원
제주해녀박물관
제주허브동산
조랑말체험공원
종달리해안도로
주상절리대
중문관광단지
지미봉
천제연폭포
천지연폭포 
초콜릿박물관
테디베어뮤지엄제주
퍼시픽리솜
피규어뮤지엄제주
한라산
한라수목원
한림공원
헬로키티아일랜드
협재해수욕장
혼인지
화순금모래 해변
황우지해안
한라생태숲
거문오름
검멀레해수욕장
금능해수욕장
다랑쉬오름
더마파크
브릭캠퍼스제주
새별오름
서귀포치유의숲
세계자동차&피아노박물관
신창풍차해안도로
용눈이오름
제주별빛누리공원
제주불빛정원
제주신산공원
제주유리의 성
제주항공우주박물관
제주현대미술관
카멜리아힐
화조원
휴애리자연생활공원
